In [1]:
%pip install aiohttp

In [53]:
import aiohttp
import asyncio
import xml.etree.ElementTree as ET
from datetime import datetime

async def fetch_quote(session, ticker):
    url = f'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/{ticker}.xml'
    async with session.get(url) as response:
        xml_data = await response.text()
        root = ET.fromstring(xml_data)
        for row in root.findall(".//row"):
            if 'LAST' in row.attrib:
                return float(row.attrib['LAST'])
        return None


async def get_moexbc(tickers_data):
    print("Начинаем расчет индекса MOEXBC")
    print(f"Количество компаний: {len(tickers_data)}")
    print("\n" + "="*85)
    print("| {:^6} | {:^10} | {:^10} | {:^10} | {:^10} | {:^10} |".format(
        "Тикер", "База кап", "База цена", "Тек. цена", "Тек. кап", "Вес"))
    print("="*85)

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_quote(session, ticker) for ticker in tickers_data]
        prices = await asyncio.gather(*tasks)

        for ticker, price in zip(tickers_data, prices):
            data = tickers_data[ticker]
            data['last_price'] = price if price else 0.0
            data['current_cap'] = data['base_cap'] * data['last_price']
            data['change'] = ((data['last_price'] - data['base_price']) / data['base_price']) * 100

            print("| {:^6} | {:>10.2f} | {:>10.2f} | {:>10.2f} | {:>10.2f} | {:>8.2%} |".format(
                ticker,
                data['base_cap'],
                data['base_price'],
                data['last_price'],
                data['current_cap'],
                data['base_cap'] / sum(t['base_cap'] for t in tickers_data.values()))
            )

    total_base = sum(data['base_cap'] for data in tickers_data.values())
    total_current = sum(data['current_cap'] for data in tickers_data.values())
    base_idx = 20038.52
    index = (total_current / total_base) * base_idx

    print("="*85 + "\n")
    print(f"Общая базовая капитализация: {total_base:,.2f}")
    print(f"Общая текущая капитализация: {total_current:,.2f}")
    print(f"Делитель индекса: {base_idx / total_base:.6f}")
    print("\n" + "="*85)
    print(f"ИТОГОВЫЙ ИНДЕКС MOEXBC: {index:,.2f}")

    return index

async def main():
    tickers_data = {
        'CHMF': {'base_cap': 110.595, 'base_price': 1435.00},
        'GAZP': {'base_cap': 643.752, 'base_price': 162.25},
        'GMKN': {'base_cap': 388.556, 'base_price': 16396.00},
        'IRAO': {'base_cap': 89.209, 'base_price': 4.069},
        'LKOH': {'base_cap': 772.998, 'base_price': 6911.50},
        'MGNT': {'base_cap': 109.185, 'base_price': 7239.00},
        'NLMK': {'base_cap': 92.078, 'base_price': 182.90},
        'NVTK': {'base_cap': 189.247, 'base_price': 1484.00},
        'PLZL': {'base_cap': 130.218, 'base_price': 10875.00},
        'ROSN': {'base_cap': 139.966, 'base_price': 600.30},
        'RUAL': {'base_cap': 95.415, 'base_price': 34.89},
        'SBER': {'base_cap': 701.917, 'base_price': 276.00},
        'SNGS': {'base_cap': 171.632, 'base_price': 28.215},
        'TATN': {'base_cap': 321.656, 'base_price': 688.20},
        'YDEX': {'base_cap': 83.504, 'base_price': 2611.00}
    }

    print("\nЗапуск расчета индекса MOEXBC")
    print("="*85)

    start_time = datetime.now()
    index = await get_moexbc(tickers_data)
    duration = (datetime.now() - start_time).total_seconds()

    print(f"\nВремя выполнения: {duration:.2f} секунд")
    print("="*85)

if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())


Запуск расчета индекса MOEXBC
Начинаем расчет индекса MOEXBC
Количество компаний: 15

| Тикер  |  База кап  | База цена  | Тек. цена  |  Тек. кап  |    Вес     |
|  CHMF  |     110.59 |    1435.00 |    1092.40 |  120813.98 |    2.74% |
|  GAZP  |     643.75 |     162.25 |     127.29 |   81943.19 |   15.93% |
|  GMKN  |     388.56 |   16396.00 |     110.78 |   43044.23 |    9.62% |
|  IRAO  |      89.21 |       4.07 |       3.55 |     316.69 |    2.21% |
|  LKOH  |     773.00 |    6911.50 |    6548.00 | 5061590.90 |   19.13% |
|  MGNT  |     109.19 |    7239.00 |    4510.00 |  492424.35 |    2.70% |
|  NLMK  |      92.08 |     182.90 |     129.40 |   11914.89 |    2.28% |
|  NVTK  |     189.25 |    1484.00 |    1167.60 |  220964.80 |    4.68% |
|  PLZL  |     130.22 |   10875.00 |    1741.80 |  226813.71 |    3.22% |
|  ROSN  |     139.97 |     600.30 |     444.25 |   62179.90 |    3.46% |
|  RUAL  |      95.42 |      34.89 |      34.44 |    3285.62 |    2.36% |
|  SBER  |     701.92 |